In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt


In [3]:
! unzip OA.zip
test = ('/content/TEST')
train = ('/content/TRAIN')

Archive:  OA.zip
   creating: TEST/Caries/
  inflating: TEST/Caries/196_2.jpg   
  inflating: TEST/Caries/197_1.jpg   
  inflating: TEST/Caries/198_2.jpg   
  inflating: TEST/Caries/199_2.jpg   
  inflating: TEST/Caries/200_2.jpg   
  inflating: TEST/Caries/201_2.jpg   
  inflating: TEST/Caries/202_2.jpg   
  inflating: TEST/Caries/203_2.jpg   
  inflating: TEST/Caries/204_2.jpg   
  inflating: TEST/Caries/205_2.jpg   
  inflating: TEST/Caries/206_2.jpg   
  inflating: TEST/Caries/206_6.jpg   
  inflating: TEST/Caries/207_2.jpg   
  inflating: TEST/Caries/208_2.jpg   
  inflating: TEST/Caries/209_2.jpg   
  inflating: TEST/Caries/210_2.jpg   
  inflating: TEST/Caries/210_6.jpg   
  inflating: TEST/Caries/211_2.jpg   
  inflating: TEST/Caries/211_6.jpg   
  inflating: TEST/Caries/212_2.jpg   
  inflating: TEST/Caries/213_2.jpg   
  inflating: TEST/Caries/214_2.jpg   
  inflating: TEST/Caries/215_2.jpg   
  inflating: TEST/Caries/216_2.jpg   
  inflating: TEST/Caries/217_2.jpg   
  infla

In [4]:
datagen = ImageDataGenerator(
    rescale=1/255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [48]:
test_data = datagen.flow_from_directory(test, target_size=(256, 256), batch_size=32, class_mode='categorical', subset='validation')

Found 80 images belonging to 2 classes.


In [49]:
train_data = datagen.flow_from_directory(train, target_size=(256, 256), batch_size=32, class_mode='categorical', subset='training')

Found 1190 images belonging to 2 classes.


In [50]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

In [51]:
b = base_model.output
b = GlobalAveragePooling2D()(b)
b = Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01))(b)
b = Dropout(0.5)(b)
predictions = Dense(len(train_data.class_indices), activation='softmax')(b)
model = Model(inputs=base_model.input, outputs=predictions)

In [54]:
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [55]:
model_eval = model.fit(train_data, epochs=10, validation_data=test_data,
                    steps_per_epoch=train_data.samples // train_data.batch_size,
                    validation_steps=test_data.samples // test_data.batch_size)

Epoch 1/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 104s 1s/step - accuracy: 0.6785 - loss: 2.8622 - val_accuracy: 0.5000 - val_loss: 218964294631424.0000
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.7500 - loss: 1.0102 - val_accuracy: 0.5000 - val_loss: 138396747956224.0000
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 581ms/step - accuracy: 0.8246 - loss: 0.7793 - val_accuracy: 0.4531 - val_loss: 46155624.0000
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8750 - loss: 0.4306 - val_accuracy: 0.6875 - val_loss: 21873412.0000
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 40s 575ms/step - accuracy: 0.8597 - loss: 0.4850 - val_accuracy: 0.4844 - val_loss: 2792035.5000
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8125 - loss: 0.8239 - val_accuracy: 0.5625 - val_loss: 2252147.7500
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 26s 582ms/step - accuracy: 0.8735 - loss: 0.3941 - val_accuracy: 0.5000 - val_loss: 74959.5391
Epoch 8/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

In [56]:
test_pred = model.predict(test_data)
y_true = test_data.classes
y_pred = np.argmax(test_pred, axis=1)

3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step


In [57]:
print(classification_report(y_true, y_pred, target_names=list(test_data.class_indices.keys())))

              precision    recall  f1-score   support

      Caries       0.50      0.03      0.05        40
  Gingivitis       0.50      0.97      0.66        40

    accuracy                           0.50        80
   macro avg       0.50      0.50      0.35        80
weighted avg       0.50      0.50      0.35        80

